## Downloading the Data

Begin with downloading the provided historic data

In [1]:
# import libraries
from urllib.request import urlretrieve
from pyspark.sql import SparkSession, functions as F


In [2]:
def create_data_folder(output_dir):
    """
    Create folders for each stage of the ETL pipeline
    :param output_dir: The base directory where the folders will be created
    """
    # set output directory
    import os
    
    # check if data directory exists, if not create it
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # create folders for each stage of the ETL pipeline
    for stage in ['landing', 'raw', 'curated', 'analysis']:
        stage_path = os.path.join(output_dir, stage)
        if not os.path.exists(stage_path):
            os.makedirs(stage_path) 


In [3]:
# create data directories
create_data_folder('../data/')

In [ ]:
# create rent_by_suburb directory
rent_by_suburb_directory = '../data/landing/rent_by_suburb'

In [8]:
# URL template for moving annual rent by suburb
URL_TEMPLATE = "https://www.dffh.vic.gov.au/moving-annual-rents-suburb-march-quarter-2023-excel"

# import os
import os

# generate output file path
output_file_path = f"{rent_by_suburb_directory}.xlsx"

# check if output file already exists
if not os.path.exists(output_file_path):
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    # download the file from the URL and save it to the output file path
    urlretrieve(URL_TEMPLATE, output_file_path)
    print(f"File downloaded and saved to {output_file_path}")
else:
    print(f"File already exists at {output_file_path}")

File downloaded and saved to ../data/landing/rent_by_suburb.xlsx
